In [1]:
import psycopg2
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import nltk.tokenize as tk
import nltk
import string
import gensim
from gensim import corpora, models
from langdetect import DetectorFactory
from langdetect import detect
from datetime import datetime

DetectorFactory.seed = 0

C:\Users\rohan_000\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Creating beerwords dictionary from entire review corpus

In [2]:
## Establishing Connection
try:
    conn = psycopg2.connect('''
    dbname='brew' user='rthakur' 
    host='capstone-brewing.cyol5m7lekm6.us-east-1.rds.amazonaws.com' 
    password='brew'
    ''')
except:
    print "Unable to connect to the database"

In [3]:
## Cleaning and Preprocessing

stop = set(stopwords.words('english'))
additional_exclude_words = [u'beer', u'one', u'like', u'well', u'really', u'very', u'it\x92s', 
                            u'it', u'beers',u'would',u'taste',u'tastes',u'get',u'i\x92m',u'quite', u'i\x92ve',
                            u'bit',u'much',u'good',u'better',u'think',u'first',u'new',u'try', u'updated']
stop |= set(additional_exclude_words)
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc, name_tokens):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = " ".join([ch for ch in doc.lower().split() if ch not in exclude])
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    tokens = tk.word_tokenize(normalized)
    tokens_with_pos = [x[0] for x in nltk.pos_tag(tokens) if x[1] in ('NN','JJ') and 
                       x[0] not in name_tokens and len(x[0])>2 and x[0] not in stop]
    return tokens_with_pos
#and wordnet.sysnet(term)
#reviews_clean = [clean(r) for r in reviews]  

In [14]:
##Compiling Documents
print str(datetime.now())
#dview = rc[:]
cur = conn.cursor()
cur.execute("""
select b.id,
       b.name,
       r.text
from wrk.beer b left join wrk.review r on b.id = r.beer_id
""")
rows = cur.fetchall()

reviews_clean = []

#def process_row(row):
#    import nltk
#    review = row[2].decode('utf-8')
#    name = row[1]
#    name_tokens = nltk.word_tokenize(name.lower())
 #   try:
#        if detect(review)=='en':
#            return clean(review.lower(), name_tokens)
#    except:
#        return

#reviews_clean = dview.map_sync(process_row, rows)

#for i, _ in enumerate(p.imap_unordered(do_work, xrange(num_tasks)), 1):
#    sys.stderr.write('\rdone {0:%}'.format(i/num_tasks))

#for _ in tqdm.tqdm(pool.imap_unordered(do_work, tasks), total=len(tasks)):
#    pass
    
i = 0
for row in rows:
    if i%10000==0:
        print i
        print str(datetime.now())
    if not row[2]:
        continue
    review = row[2].decode('utf-8')
    name = row[1]
    name_tokens = nltk.word_tokenize(name.lower())
    try:
        if detect(review)=='en':
            reviews_clean.append(clean(review.lower(), name_tokens))
    except:
        continue
    i+=1
print str(datetime.now())

2017-07-31 07:48:21.261000
0
2017-07-31 07:50:11.779000
10000
2017-07-31 07:53:58.229000
20000
2017-07-31 07:57:32.547000
30000
2017-07-31 08:01:26.506000
40000
2017-07-31 08:04:54.122000
50000
2017-07-31 08:08:46.196000
60000
2017-07-31 08:13:08.419000
70000
2017-07-31 08:17:00.284000
80000
2017-07-31 08:20:15.591000
90000
2017-07-31 08:23:47.684000
100000
2017-07-31 08:28:20.785000
110000
2017-07-31 08:39:01.466000
120000
2017-07-31 08:50:38.208000
130000
2017-07-31 09:00:52.565000
140000
2017-07-31 09:04:47.526000
150000
2017-07-31 09:08:36.249000
160000
2017-07-31 09:13:04.655000
170000
2017-07-31 09:17:43.891000
180000
2017-07-31 09:22:02.827000
190000
2017-07-31 09:25:57.747000
200000
2017-07-31 09:29:12.783000
210000
2017-07-31 09:32:35.909000
220000
2017-07-31 09:36:05.258000
230000
2017-07-31 09:39:44.548000
240000
2017-07-31 09:43:32.424000
250000
2017-07-31 09:47:45.868000
260000
2017-07-31 09:51:19.702000
270000
2017-07-31 09:54:46.511000
280000
2017-07-31 09:58:22.543000
2

In [15]:
print len(reviews_clean)

982568


In [17]:
# Creating the term dictionary of our corpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(reviews_clean)

# Saving file
dictionary.save('../../data/beerwords.dict')

In [ ]:
import pickle

with open('../../data/reviews_clean', 'wb') as fp:
    pickle.dump(reviews_clean, fp)

## Saving the larger beer model and corpus 

In [4]:
## Establishing Connection

dictionary = corpora.Dictionary.load('../../data/beerwords.dict')
try:
    conn = psycopg2.connect('''
    dbname='brew' user='rthakur' 
    host='capstone-brewing.cyol5m7lekm6.us-east-1.rds.amazonaws.com' 
    password='brew'
    ''')
except:
    print "Unable to connect to the database"

In [5]:
cur = conn.cursor()
corpus = []

for x in range(4611):
    if x%500==0:
        print x
        print str(datetime.now())
    cur.execute("""
    select b.id,
       b.name,
       r.text
    from wrk.beer b left join wrk.review r on b.id = r.beer_id
    where r.beer_id={0}
    """.format(x+1))
    rows = cur.fetchall()
    reviews_clean = []
    for row in rows:
        review = row[2].decode('utf-8')
        name = row[1]
        name_tokens = nltk.word_tokenize(name.lower())
        try:
            if detect(review)=='en':
                rev_clean = clean(review.lower(), name_tokens)
                reviews_clean.append(u' '.join(rev_clean))
        except:
            continue
    corpus += [dictionary.doc2bow(reviews_clean)] 

0
2017-08-01 01:11:33.500000
500
2017-08-01 01:53:15.749000
1000
2017-08-01 02:31:47.596000
1500
2017-08-01 03:17:04.866000
2000
2017-08-01 03:45:19.887000
2500
2017-08-01 04:30:07.306000
3000
2017-08-01 05:03:27.176000
3500
2017-08-01 05:51:38.146000
4000
2017-08-01 06:22:06.722000
4500
2017-08-01 06:49:50.364000


In [6]:
corpora.MmCorpus.serialize('../../data/beercorpus.mm', corpus)
#import pickle

#with open('../../data/corpus', 'wb') as fp:
#    pickle.dump(corpus, fp)

In [7]:
beer_model =  models.LdaModel(corpus, id2word=dictionary, num_topics=200)
#print corpus

In [8]:
beer_model.save('../../data/beer_model')